In [13]:
import pandas as pd
from unicodedata2 import normalize
import os
import glob
import re

In [78]:
# Função para normalizar nomes de colunas, evitando erros futuros em salvamentos
def normalizar_colunas(dataframe):
    lista_colunas = []
    for coluna in dataframe.columns:
        nova_coluna = normalize('NFKD', coluna).encode('ascii', 'ignore').decode('utf-8').replace(' ', '_')
        lista_colunas.append(nova_coluna)
    dataframe.columns = lista_colunas
    return dataframe

# Função para remover partidas que não devam constar no arquivo final por estarem com o código errado
def remover_por_codigo():
    files = glob.glob(r"..\\Excel\\*.xlsx")
    for file in files:
        excel_file_name = file.split('\\')[-1]
        # Verifica se "REMOVER" está no nome do arquivo e se o código "TxxBxx" também está presente ("x" são números)
        if "REMOVER" in excel_file_name.upper() and re.search(r'T\d{2}B\d{2}', excel_file_name):
            codigo_partida = re.search(r'T\d{2}B\d{2}', excel_file_name).group(0)
            for csv_file in glob.glob(r"..\\CSV\\01.bronze\\*.csv"):
                try:
                    csv_file_name = csv_file.split('\\')[-1]
                    file_path = f"..\\CSV\\01.bronze\\{csv_file_name}"
                    df = pd.read_csv(file_path)
                
                    df_filtered = df[~df.apply(lambda row: row.astype(str).str.contains(codigo_partida).any(), axis=1)]
                    
                    df_filtered.to_csv(file_path, index=False)
                    os.remove(file)
                except Exception as e:
                    if str(e).__contains__("Permission denied"):
                        print("Feche o arquivo antes de salvar!")
                    else:
                        print(e)

#TODO Alterar forma de salvamento para quando o valor encontrado já existir no arquivo bronze
def salvamento_csv(dataframe, file_name, excel_file_name):
    file_path = f"..\\CSV\\01.bronze\\bronze_{file_name}.csv"
    if os.path.exists(file_path):
        dataframe.to_csv(file_path, mode='a', header=False, index=False)
        print(f"Anexando '{excel_file_name}' a 'bronze_{file_name}.csv'.")
    else:
        dataframe.to_csv(file_path, mode='w', header=True, index=False)
        print(f"Criando 'bronze_{file_name}.csv' a partir de '{excel_file_name}'.")

In [114]:
remover_por_codigo()


files = glob.glob(r"..\\Excel\\Sumula_Teste_T[0-9][0-9]B[0-9][0-9].xlsx")
for file in files:
    try:
        csv_indivudual = pd.read_csv(next(file for file in glob.glob(r"..\\CSV\\01.bronze\\*.csv") if "individual" in file.lower()))
        csv_times = pd.read_csv(next(file for file in glob.glob(r"..\\CSV\\01.bronze\\*.csv") if "times" in file.lower()))
        csv_info = pd.read_csv(next(file for file in glob.glob(r"..\\CSV\\01.bronze\\*.csv") if "info" in file.lower()))
        excel_file_name = file.split('\\')[-1]
        arquivo_completo = pd.read_excel(file, sheet_name=None, engine="calamine")
    except Exception as e:
                if str(e).__contains__("Permission denied"):
                    print("Feche o arquivo antes de salvar!")
                else:
                    print(e)

    df_individual = arquivo_completo["Individual"]
    df_times = arquivo_completo["Times"]
    df_info = arquivo_completo["Informações"]

    normalizar_colunas(df_individual)
    normalizar_colunas(df_times)
    normalizar_colunas(df_info)

    codigo_arquivo = re.search(r'T\d{2}B\d{2}', excel_file_name).group(0)
    codigo_baba_ind = df_individual["Baba"].unique()
    qtd_codigo_baba_ind = len(codigo_baba_ind)
    codigo_baba_times = df_times["Baba"].unique()
    qtd_codigo_baba_times = len(codigo_baba_times)
    codigo_validacao = df_info["Baba"][0]

    for valor in codigo_baba_ind:
        if valor != codigo_validacao:
            print(f"Há códigos do baba diferentes em {excel_file_name} na aba 'Individual'.")
        
    for valor in codigo_baba_times:
        if valor != codigo_validacao:
            print(f"Há códigos do baba diferentes em {excel_file_name} na aba 'Times'.")

    if (qtd_codigo_baba_ind == 1) and (qtd_codigo_baba_times == 1):
        if codigo_baba_ind == codigo_baba_times == codigo_validacao == codigo_arquivo:
            try:
                # print(codigo_arquivo)
                if codigo_arquivo in csv_info.iloc[:,0].unique():
                    # print(f"O código encontrado é {codigo_arquivo}")
                    csv_indivudual = csv_indivudual[~csv_indivudual.apply(lambda row: row.astype(str).str.contains(codigo_arquivo).any(), axis=1)]
                    csv_times = csv_times[~csv_times.apply(lambda row: row.astype(str).str.contains(codigo_arquivo).any(), axis=1)]
                    csv_info = csv_info[~csv_info.apply(lambda row: row.astype(str).str.contains(codigo_arquivo).any(), axis=1)]
                    csv_indivudual.to_csv(next(file for file in glob.glob(r"..\\CSV\\01.bronze\\*.csv") if "individual" in file.lower()), index=False)
                    csv_times.to_csv(next(file for file in glob.glob(r"..\\CSV\\01.bronze\\*.csv") if "times" in file.lower()), index=False)
                    csv_info.to_csv(next(file for file in glob.glob(r"..\\CSV\\01.bronze\\*.csv") if "info" in file.lower()), index=False)
                salvamento_csv(df_individual, "desempenho_individual", excel_file_name)
                salvamento_csv(df_times, "desempenho_times", excel_file_name)
                salvamento_csv(df_info, "info_partidas", excel_file_name)
            except Exception as e:
                if str(e).__contains__("Permission denied"):
                    print("Feche o arquivo antes de salvar!")
                else:
                    print(e)
        else:
            print(f"Há códigos de baba diferentes em {excel_file_name}.")
    else:
            print(f"Há códigos de baba diferentes em {excel_file_name}.")


Anexando 'Sumula_Teste_T01B01.xlsx' a 'bronze_desempenho_individual.csv'.
Anexando 'Sumula_Teste_T01B01.xlsx' a 'bronze_desempenho_times.csv'.
Anexando 'Sumula_Teste_T01B01.xlsx' a 'bronze_info_partidas.csv'.
Anexando 'Sumula_Teste_T01B02.xlsx' a 'bronze_desempenho_individual.csv'.
Anexando 'Sumula_Teste_T01B02.xlsx' a 'bronze_desempenho_times.csv'.
Anexando 'Sumula_Teste_T01B02.xlsx' a 'bronze_info_partidas.csv'.
Há códigos do baba diferentes em Sumula_Teste_T01B03.xlsx na aba 'Individual'.
Há códigos do baba diferentes em Sumula_Teste_T01B03.xlsx na aba 'Times'.
Há códigos de baba diferentes em Sumula_Teste_T01B03.xlsx.
Anexando 'Sumula_Teste_T01B04.xlsx' a 'bronze_desempenho_individual.csv'.
Anexando 'Sumula_Teste_T01B04.xlsx' a 'bronze_desempenho_times.csv'.
Anexando 'Sumula_Teste_T01B04.xlsx' a 'bronze_info_partidas.csv'.


In [65]:
csv_info = pd.read_csv(next(file for file in glob.glob(r"..\\CSV\\01.bronze\\*.csv") if "info" in file.lower()))
print(csv_info.iloc[:,0])

0     T01B03
1     T01B04
2     T01B03
3     T01B04
4     T01B03
5     T01B04
6     T01B03
7     T01B04
8     T01B03
9     T01B04
10    T01B01
11    T01B01
12    T01B01
13    T01B04
14    T01B01
15    T01B04
16    T01B01
17    T01B04
18    T01B01
19    T01B02
20    T01B04
Name: Baba, dtype: object


In [63]:
codigo_arquivo

'T01B04'

In [90]:
csv_info.iloc[:,0].unique()

array(['T01B03', 'T01B04', 'T01B01', 'T01B02'], dtype=object)

In [18]:
df_individual

,Baba,Jogador,Time,Gols,Assistencias
0,T01B04,José,Vermelho,8,3
1,T01B04,Renato,Vermelho,2,4
2,T01B04,Matheus,Vermelho,0,2
3,T01B04,Arthur,Vermelho,1,1
4,T01B04,Heitor,Vermelho,1,0
5,T01B04,Isaac,Azul,3,0
6,T01B04,Nicolas,Azul,0,3
7,T01B04,Murilo,Azul,1,1
8,T01B04,Vicente,Azul,0,0
9,T01B04,Paulo,Azul,1,1


In [70]:
df_times

,Baba,Time,Campeao,Vitorias,Empates,Derrotas
0,T01B04,Vermelho,1,5,2,1
1,T01B04,Azul,0,2,3,3
2,T01B04,Verde,0,1,3,4


In [71]:
df_info

,Baba,Data,Inicio,Fim,Arbitro,Nota,Campo,Coordenadas
0,T01B04,2024-08-17,16:00:00,19:00:00,Carlinho Apito Longo,8,Complexo Poliesportivo da UEFS,"-12.197253215204276, -38.97078135645937"
